In [2]:
b2.get_file('modules/auxiliary_functions.py')
!pip install 'powerlaw'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 31.6 MB/s eta 0:00:00


In [4]:
# can do this when i have converted the notebooks to .py files
# import auxiliary_functions
import auxiliary_functions as af
import bmll2 as b2

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation

from statsmodels.sandbox.stats.runs import runstest_1samp 
import powerlaw
import itertools
import pylab
import scipy.stats
from scipy.optimize import curve_fit

import shutil
from pathlib import Path
import warnings

In [5]:
b2.get_file('top100_tickers.csv')
top100_tickers = pd.read_csv('top100_tickers.csv')

In [6]:
top100_tickers = top100_tickers['0'].tolist()

In [7]:
files = b2.list_files(path = 'top_100(Volume)')

for f in files:
    b2.get_file(f'top_100(Volume)/{f}')
    
def load_stock(csv):
    df = pd.read_csv(csv, parse_dates = ['DateTime', 'Date'])
    df = df.rename(columns = {'Ticker': 'RIC'})
    return df
    
stocks = {
    ticker: load_stock(f'{ticker}.csv')
    for ticker in top100_tickers
}

In [8]:
def one_sided_runs_test(sample, runs_correction = 0):
    sample = pd.Series(sample)
    values = sample.unique()

    if (len(values) < 2):
        return 'Only 1 unique value present'
        
    a = values[0]
    b = values[1]

    N_a  = len(sample[sample == a])
    N_b  = len(sample[sample == b])
    N    = N_a + N_b
    mu   = ((2 * N_a * N_b) / N) + 1
    runs = itertools.groupby(sample)
    R    = sum(1 for _ in runs)
    R_corrected = R + runs_correction

    sigma = np.sqrt((2 * N_a * N_b * (2 * N_a * N_b - N)) / (N ** 2 * (N - 1)))
    z     = (R_corrected - mu) / sigma

    p_value = scipy.stats.norm.cdf(z)

    return (z, p_value, R)

In [11]:
N                   = 200
trader_distribution = 'power' # or homogenous
alpha               = 2

In [12]:
%%time
stocks_trade_sequences    = [] # will be a 3 deep list. 40 stocks, all days, < 10 sequences per day, indexes of trades for ST traders
stocks_p_vals             = [] # will be a 3 deep list. 40 stocks, all days, < 10 p vals per day, a single value per ST trader
stocks_run_lengths        = [] # will be a 3 deep list. 40 stocks, all days, < 10 traders per day, run lengths of each metaorder
stocks_total_volumes      = [] # will be a 2 deep list, 40 stocks, all days, total volume traded per day of each stock
stocks_percentage_STs     = [] 
stocks_percentage_STs_vol = []
stocks_percentage_STs_num = []

for ric, stock_data in stocks.items():

    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data['Year'] = stock_data['Date'].dt.year

    for year, df_year in stock_data.groupby('Year'):
    
        trader_trade_sequences      = [] # will be a 2 deep list. all days, < 10 sequences per day, indexes of trades for ST traders
        trader_p_vals               = [] # will be a 2 deep list. all days, < 10 p vals per day, a single value per ST trader
        trader_run_lengths          = [] # will be a 2 deep list. all days, < 10 traders per day, run lengths of each metaorder
        trader_total_volumes        = [] # will be a 1 deep list. all days, total volume traded per day
        trader_percentage_STs       = []
        trader_percentage_STs_vol   = []
        trader_percentage_STs_num   = []
    
        f = af.trader_participation(N = N, method = trader_distribution, alpha = alpha, f_min = 1, f_max = stock_data.shape[0], seed = 1)
        c = af.cumulative_probs(f)
    
        if stock_data.empty:
            continue
    
        output = af.orders(N = N, trades = df_year, cumulative_probs = c)
        
        STs_volume = 0
        STs_num    = 0
        for n in range(N):
    
            trader_n_trades = df_year.iloc[output[n], ]
    
            if (trader_n_trades.shape[0] <= 2):
                continue
    
            if (trader_n_trades['Trade Sign'].nunique() < 2):
                continue
    
            day_breaks = 0
            prev_date  = None
            prev_sign  = None
            
            for idx, row in trader_n_trades.iterrows():
                current_date = row['Date']
                current_sign = row['Trade Sign']
                
                # If we've moved to a new day and the sign is THE SAME from end of previous day
                if prev_date is not None and current_date != prev_date and current_sign == prev_sign:
                    day_breaks += 1
                
                prev_date = current_date
                prev_sign = current_sign
    
            runs_test = one_sided_runs_test(trader_n_trades['Trade Sign'], runs_correction = day_breaks)
            p_val     = runs_test[1]
    
            # we need to decide on an appropriate p value here. 1% seems too strict
            if p_val <= 0.01:
    
                STs_volume = STs_volume + sum(trader_n_trades['Volume'])
                STs_num    = STs_num + trader_n_trades.shape[0]
                trader_trade_sequences.append(trader_n_trades)
                trader_p_vals.append(p_val)
                
                grouped_trade_signs = itertools.groupby(trader_n_trades['Trade Sign'])
                
                for key, group in grouped_trade_signs:
                    trader_run_lengths.append(len(list(group)))
    
            #STs_percentage_vol = round((STs_volume / sum(stock_data['Volume'])) * 100, 3)
            #STs_percentage_num = round((STs_num / stock_data.shape[0]) * 100, 3)
                                                  
            #trader_trade_sequences.append(days_trade_sequences)
            #trader_p_vals.append(days_p_vals)
            #trader_sequence_run_lengths.append(days_run_lengths)
            trader_total_volumes.append(sum(stock_data['Volume']))
            #trader_percentage_STs.append(round((len(trader_trade_sequences) / N) * 100, 3))
            #trader_percentage_STs_vol.append(STs_percentage_vol)
            #trader_percentage_STs_num.append(STs_percentage_num)

        STs_percentage_vol = round((STs_volume / sum(stock_data['Volume'])) * 100, 3)
        STs_percentage_num = round((STs_num / stock_data.shape[0]) * 100, 3)
        
        stocks_trade_sequences.append(trader_trade_sequences)
        stocks_p_vals.append(trader_p_vals)
        stocks_run_lengths.append(trader_run_lengths)
        stocks_total_volumes.append(trader_total_volumes)
        stocks_percentage_STs.append(round((len(trader_trade_sequences) / N) * 100, 3))
        stocks_percentage_STs_vol.append(STs_percentage_vol)
        stocks_percentage_STs_num.append(STs_percentage_num)
        print('done with', ric, 'for', year)


stocks_percentage_STs     = np.array(stocks_percentage_STs)
stocks_percentage_STs_vol = np.array(stocks_percentage_STs_vol)
stocks_percentage_STs_num = np.array(stocks_percentage_STs_num)

STs_percentage = stocks_percentage_STs[stocks_percentage_STs > 0]
STs_vols       = stocks_percentage_STs_vol[stocks_percentage_STs_vol > 0]
STs_nums       = stocks_percentage_STs_num[stocks_percentage_STs_num > 0]

ST_df          = pd.DataFrame({'% STs' : STs_percentage, 'STs volume' : STs_vols, 'STs number' : STs_nums})
ST_df.to_csv(f'ST_df_{trader_distribution}_{N}.csv', index = False)
b2.put_file(f'ST_df_{trader_distribution}_{N}.csv', 'test_data')   

# takes about 4 hours to run

done with LAB for 2023
done with LAB for 2025
done with PPH for 2023
done with PPH for 2024
done with PPH for 2025
done with FSR for 2023
done with FSR for 2024
done with FSR for 2025
done with OMU for 2023
done with OMU for 2024
done with OMU for 2025
done with WHL for 2023
done with WHL for 2025
done with SSW for 2023
done with SSW for 2024
done with SSW for 2025
done with FTB for 2023
done with FTB for 2024
done with FTB for 2025
done with ORN for 2023
done with ORN for 2024
done with ORN for 2025
done with PAN for 2023
done with PAN for 2024
done with PAN for 2025
done with GLN for 2023
done with GLN for 2024
done with GLN for 2025
done with PIK for 2023
done with PIK for 2024
done with PIK for 2025
done with RDF for 2023
done with RDF for 2024
done with RDF for 2025
done with OUT for 2023
done with OUT for 2024
done with OUT for 2025
done with SOL for 2023
done with SOL for 2024
done with SOL for 2025
done with GRT for 2023
done with GRT for 2024
done with GRT for 2025
done with M

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide



done with CPR for 2023
done with CPR for 2024


/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide



done with CPR for 2025


...

done with RBO for 2024


/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide



done with RBO for 2025
done with SPG for 2023
done with SPG for 2024
done with SPG for 2025
done with WBC for 2024
done with WBC for 2025
done with LTE for 2023
done with LTE for 2024
done with LTE for 2025
done with SSS for 2023
done with SSS for 2024
done with SSS for 2025
done with AGL for 2023
done with AGL for 2024
done with BID for 2023
done with BID for 2024
done with BID for 2025
done with PPE for 2023
done with PPE for 2024
done with PPE for 2025
done with AFT for 2023
done with AFT for 2024
done with AFT for 2025
done with TKG for 2023
done with TKG for 2024
done with TKG for 2025
done with EQU for 2023
done with EQU for 2024
done with EQU for 2025
done with YRK for 2023


/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide

/tmp/ipykernel_1039/1905302829.py:20: RuntimeWarning:

divide by zero encountered in scalar divide



done with YRK for 2024
done with YRK for 2025
done with MSP for 2023
done with MSP for 2024
done with MSP for 2025
done with TGA for 2024
done with TGA for 2025
done with AEG for 2023
done with AEG for 2024
done with AEG for 2025
done with OMN for 2023
done with OMN for 2024
done with OMN for 2025
done with BTN for 2023
done with BTN for 2024
done with BTN for 2025
done with AFE for 2023
done with AFE for 2024
done with AFE for 2025
done with AVI for 2023
done with AVI for 2024
done with AVI for 2025
done with EXX for 2023
done with EXX for 2024
done with EXX for 2025
done with RNI for 2023
done with RNI for 2024
done with RNI for 2025
done with BHG for 2023
done with BHG for 2024
done with BHG for 2025
done with BOX for 2024
done with BOX for 2025
done with ANH for 2023
done with ANH for 2025
CPU times: user 3h 6min 2s, sys: 9.5 s, total: 3h 6min 12s
Wall time: 3h 6min 10s


In [10]:
def extract_ST_run_lengths(stock_data, N = 100, trader_distribution = 'power', alpha = 2, p_threshold = 0.01):
    """
    Extracts metaorder run lengths L from statistically identified
    splitting traders (STs) for a given stock and time window.
    
    Returns
    -------
    run_lengths : list of int
        Metaorder lengths pooled across ST traders
    """
    run_lengths = []
    f = af.trader_participation(N = N, method = trader_distribution, alpha =  alpha, f_min = 1 ,f_max = stock_data.shape[0], seed = 1)
    c = af.cumulative_probs(f)
    output = af.orders(N = N, trades = stock_data, cumulative_probs = c)

    for n in range(N):
        trader_trades = stock_data.iloc[output[n]]

        # Too few trades → useless
        if trader_trades.shape[0] <= 2:
            continue

        # No sign variation → no runs test
        if trader_trades['Trade Sign'].nunique() < 2:
            continue

        # Correct runs across day boundaries
        day_breaks = 0
        prev_date  = None
        prev_sign  = None

        for idx, row in trader_trades.iterrows():
            if prev_date is not None:
                if row['Date'] != prev_date and row['Trade Sign'] == prev_sign:
                    day_breaks += 1
            prev_date = row['Date']
            prev_sign = row['Trade Sign']

        z, p_val, R = one_sided_runs_test(trader_trades['Trade Sign'], runs_correction = day_breaks)

        # Identify splitting traders
        if p_val <= p_threshold:
            for key, group in itertools.groupby(trader_trades['Trade Sign']):
                run_lengths.append(len(list(group)))

    return run_lengths

In [35]:
%%time
N                   = 10
trader_distribution = 'homogenous'
alpha               = 2

identifier = f'{trader_distribution}_{N}'

for ticker, stock_data in stocks.items():

    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    stock_data['Year'] = stock_data['Date'].dt.year

    for year, df_year in stock_data.groupby('Year'):

        if df_year.shape[0] < 1000:
            continue  # too small for power laws

        L = extract_ST_run_lengths(df_year, N = N, trader_distribution = trader_distribution, alpha = alpha)
        signs = df_year['Trade Sign']
        
        if len(L) == 0:
            continue
        
        out = pd.DataFrame({'L' : L})
        out.to_csv(f'{ticker}_{trader_distribution}_{N}_run_lengths_yearly_{year}.csv', index = False)
        b2.put_file(f'{ticker}_{trader_distribution}_{N}_run_lengths_yearly_{year}.csv', 'test_data')

        signs = pd.DataFrame({'Trade Sign' : signs})
        signs.to_csv(f'{ticker}_{trader_distribution}_{N}_trade_signs_{year}.csv', index = False)
        b2.put_file(f'{ticker}_{trader_distribution}_{N}_trade_signs_{year}.csv', 'test_data')
        
    print(f'saved yearly run lengths for {ticker}')

# takes about 2 hours

saved yearly run lengths for LAB
saved yearly run lengths for PPH
saved yearly run lengths for FSR
saved yearly run lengths for OMU
saved yearly run lengths for WHL
saved yearly run lengths for SSW
saved yearly run lengths for FTB
saved yearly run lengths for ORN
saved yearly run lengths for PAN
saved yearly run lengths for GLN
saved yearly run lengths for PIK
saved yearly run lengths for RDF
saved yearly run lengths for OUT
saved yearly run lengths for SOL
saved yearly run lengths for GRT
saved yearly run lengths for MTN
saved yearly run lengths for IMP
saved yearly run lengths for LHC
saved yearly run lengths for SAP
saved yearly run lengths for SLM
saved yearly run lengths for SAC
saved yearly run lengths for TRU
saved yearly run lengths for OPA
saved yearly run lengths for SBK
saved yearly run lengths for ABG
saved yearly run lengths for VKE
saved yearly run lengths for NTC
saved yearly run lengths for CPR
saved yearly run lengths for SSU
saved yearly run lengths for DRD
saved year

In [36]:
%%time
files = b2.list_files(path = 'test_data')

files = [f for f in files if (f'{identifier}_run_lengths' in f) or (f'{identifier}_trade_signs' in f)]
for f in files:
    b2.get_file(f'test_data/{f}')

# takes about 2 minutes to load

CPU times: user 11.3 s, sys: 1.59 s, total: 12.9 s
Wall time: 2min 25s


In [37]:
%%time
yearly_run_lengths_2023 = {}
yearly_run_lengths_2024 = {}
yearly_run_lengths_2025 = {}

for ticker in top100_tickers:

    try:
        yearly_run_lengths_2023[ticker] = pd.read_csv(f'{ticker}_{identifier}_run_lengths_yearly_2023.csv')
    except FileNotFoundError:
        print (f'{ticker}_{identifier}_run_lengths_yearly_2023.csv not found')
        pass
        
    try:
        yearly_run_lengths_2024[ticker] = pd.read_csv(f'{ticker}_{identifier}_run_lengths_yearly_2024.csv')
    except FileNotFoundError:
        print (f'{ticker}_{identifier}_run_lengths_yearly_2024.csv not found')
        pass

    try:
        yearly_run_lengths_2025[ticker] = pd.read_csv(f'{ticker}_{identifier}_run_lengths_yearly_2025.csv')
    except FileNotFoundError:
        print (f'{ticker}_{identifier}_run_lengths_yearly_2025.csv not found')
        pass


LAB_homogenous_10_run_lengths_yearly_2023.csv not found
LAB_homogenous_10_run_lengths_yearly_2024.csv not found
LAB_homogenous_10_run_lengths_yearly_2025.csv not found
ORN_homogenous_10_run_lengths_yearly_2023.csv not found
ORN_homogenous_10_run_lengths_yearly_2024.csv not found
ORN_homogenous_10_run_lengths_yearly_2025.csv not found
OPA_homogenous_10_run_lengths_yearly_2023.csv not found
OPA_homogenous_10_run_lengths_yearly_2024.csv not found
CPR_homogenous_10_run_lengths_yearly_2023.csv not found
CPR_homogenous_10_run_lengths_yearly_2025.csv not found
KP2_homogenous_10_run_lengths_yearly_2023.csv not found
KP2_homogenous_10_run_lengths_yearly_2024.csv not found
KP2_homogenous_10_run_lengths_yearly_2025.csv not found
DIB_homogenous_10_run_lengths_yearly_2023.csv not found
DIB_homogenous_10_run_lengths_yearly_2025.csv not found
MDI_homogenous_10_run_lengths_yearly_2023.csv not found
MDI_homogenous_10_run_lengths_yearly_2024.csv not found
MDI_homogenous_10_run_lengths_yearly_2025.csv no

In [38]:
%%time
trade_signs_2023 = {}
trade_signs_2024 = {}
trade_signs_2025 = {}

for ticker in top100_tickers:

    try:
        trade_signs_2023[ticker] = pd.read_csv(f'{ticker}_{identifier}_trade_signs_2023.csv')
    except FileNotFoundError:
        print (f'{ticker}_{identifier}_trade_signs_2023.csv not found')
        pass
        
    try:
        trade_signs_2024[ticker] = pd.read_csv(f'{ticker}_{identifier}_trade_signs_2024.csv')
    except FileNotFoundError:
        print (f'{ticker}_{identifier}_trade_signs_2024.csv not found')
        pass

    try:
        trade_signs_2025[ticker] = pd.read_csv(f'{ticker}_{identifier}_trade_signs_2025.csv')
    except FileNotFoundError:
        print (f'{ticker}_{identifier}trade_signs_2025.csv not found')
        pass


LAB_homogenous_10_trade_signs_2023.csv not found
LAB_homogenous_10_trade_signs_2024.csv not found
LAB_homogenous_10trade_signs_2025.csv not found
ORN_homogenous_10_trade_signs_2023.csv not found
ORN_homogenous_10_trade_signs_2024.csv not found
ORN_homogenous_10trade_signs_2025.csv not found
OPA_homogenous_10_trade_signs_2023.csv not found
OPA_homogenous_10_trade_signs_2024.csv not found
CPR_homogenous_10_trade_signs_2023.csv not found
CPR_homogenous_10trade_signs_2025.csv not found
KP2_homogenous_10_trade_signs_2023.csv not found
KP2_homogenous_10_trade_signs_2024.csv not found
KP2_homogenous_10trade_signs_2025.csv not found
DIB_homogenous_10_trade_signs_2023.csv not found
DIB_homogenous_10trade_signs_2025.csv not found
MDI_homogenous_10_trade_signs_2023.csv not found
MDI_homogenous_10_trade_signs_2024.csv not found
MDI_homogenous_10trade_signs_2025.csv not found
CCD_homogenous_10_trade_signs_2023.csv not found
CCD_homogenous_10_trade_signs_2024.csv not found
JBL_homogenous_10_trade_si

In [39]:
def sign_autocorrelation(signs, max_lag):
    """
    Computes the sample sign autocorrelation C(tau) for tau = 1,...,max_lag
    
    Parameters
    ----------
    signs : array-like
        Sequence of trade signs (+1 / -1)
    max_lag : int
        Maximum lag tau to compute
    
    Returns
    -------
    taus : ndarray
        Array of lag values
    C : ndarray
        Sample autocorrelation values
    """
    signs = np.asarray(signs)
    N = len(signs)

    taus = np.arange(1, max_lag + 1)
    C = np.empty(max_lag)

    for i, tau in enumerate(taus):
        C[i] = np.mean(signs[:-tau] * signs[tau:])

    return taus, C

In [40]:
def select_powerlaw_range(taus, C, min_points = 1000):

    mask = (C > 0) & (np.isfinite(C))
    taus = taus[mask]
    C   = C[mask]

    log_tau = np.log(taus)
    log_C   = np.log(C)

    best       = None
    best_score = -np.inf

    for i in range(len(taus) - min_points): # stops it from going to far so that we always have min_points in the window, specifically at the end 
        for j in range(i + min_points, len(taus)): # same reason but specifically at the begining
            x = log_tau[i:j] 
            y = log_C[i:j] 
            slope, intercept = np.polyfit(x, y, 1)
            y_hat = intercept + (slope * x)
            r2    = 1 - np.sum((y - y_hat) ** 2) / np.sum((y - y.mean()) ** 2)

            if r2 > best_score:
                best_score = r2
                best = (i, j, slope)
                
    return best 

def select_powerlaw_range_fast(taus, C, min_points = 10000):

    mask = (C > 0) & np.isfinite(C)
    taus, C = taus[mask], C[mask]

    log_tau = np.log(taus)
    log_C   = np.log(C)

    N = len(log_tau)

    # cumulative sums
    # x is the log taus
    # y is the log C
    Sx  = np.cumsum(log_tau)
    Sy  = np.cumsum(log_C)
    Sxx = np.cumsum(log_tau ** 2)
    Syy = np.cumsum(log_C ** 2)
    Sxy = np.cumsum(log_tau * log_C)

    best_r2 = -np.inf
    best_i  = None

    j = N  # fixed right endpoint (tail fit)

    for i in range(N - min_points):
        
        n = j - i

        # gonna be working backwards here. We already precomputed the cumulative sum of the entire series we just need to subtract
        # the part thats no longer in it. It starts off on the largest window and makes it smaller each time. With this method we fix
        # the left endpoint of the window so that we only need to do a O(N) search and not a O(N^2) search. 
        sum_x  = Sx[j - 1]  - (Sx[i - 1]  if i > 0 else 0)
        sum_y  = Sy[j - 1]  - (Sy[i - 1]  if i > 0 else 0)
        sum_xx = Sxx[j - 1] - (Sxx[i - 1] if i > 0 else 0)
        sum_yy = Syy[j - 1] - (Syy[i - 1] if i > 0 else 0)
        sum_xy = Sxy[j - 1] - (Sxy[i - 1] if i > 0 else 0)

        denom = n * sum_xx - sum_x ** 2
        if denom == 0:
            continue

        slope = ((n * sum_xy) - (sum_x * sum_y)) / denom
        intercept = (sum_y - (slope * sum_x)) / n

        ss_tot = sum_yy - (sum_y ** 2)/n
        ss_res = sum_yy + (slope ** 2) * sum_xx + (n * intercept ** 2) - (2 * slope * sum_xy) - (2 * intercept * sum_y) + (2 * slope * intercept * sum_x)

        r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else -np.inf

        if r2 > best_r2:
            best_r2 = r2
            best_i  = i
            best_slope = slope

    return best_i, N - 1, best_slope


In [41]:
def powerlaw_fit(tau, C_0, gamma):
    
    return C_0 * (tau ** (- gamma))
    

In [42]:
def estimate_gamma_loglog(taus, C):

    x = np.log(taus)
    y = np.log(C)

    slope, intercept = np.polyfit(x, y, 1)
    gamma = -slope
    C_0 = np.exp(intercept)

    return C_0, gamma
    

In [43]:
def estimate_gamma(taus, C, min_points, nbins):

    mask = (C > 0) & (np.isfinite(C))
    taus = taus[mask]
    C    = C[mask]
    
    i, j, slope     = select_powerlaw_range_fast(taus, C, min_points)
    gamma_nlls_init = -1 * slope

    tau_min = taus[i]
    tau_max = taus[j]

    mask_fit = (taus >= tau_min) & (taus <= tau_max) & (np.isfinite(C))
    taus     = taus[mask_fit]
    C        = C[mask_fit]
    
    #taus_smooth, C_smooth = log_bin(taus, C, nbins = nbins)
    
    #C_0_init = C_smooth[0] if len(C_smooth) > 0 else 1.0
    #popt, pcov      = curve_fit(powerlaw_fit, taus_smooth, C_smooth, p0 = (C_0_init, gamma_nlls_init), maxfev = 10000) #taus[i:j] C[i:j] C[i]
    #C_0, gamma_nlls = popt

    C_0_init        = C[0] if len(C) > 0 else 1.0
    C_0, gamma_nlls = estimate_gamma_loglog(taus, C)

    return {'C_0' : C_0, 'gamma_nlls' : gamma_nlls}#, 'taus_smooth' : taus_smooth, 'C_smoth' : C_smooth}
    

In [44]:
def estimate_alpha(run_lengths):
    
    fit = powerlaw.Fit(run_lengths, discrete = True)
    
    # The powerlaw package gives alpha for CCDF P_>(L) ~ L^(-alpha)
    # For PDF P(L) ~ L^(-alpha-1), we have the same alpha
    alpha = fit.alpha - 1  # Adjust for PDF vs CCDF convention
    xmin  = fit.xmin
    
    return alpha, xmin
    

In [45]:
%%time
results = []

for year, L_dict, trade_dict in [
    (2023, yearly_run_lengths_2023, trade_signs_2023),
    (2024, yearly_run_lengths_2024, trade_signs_2024),
    (2025, yearly_run_lengths_2025, trade_signs_2025),
]:
    print(f'\033[91mProcessing year: {year}\033[0m')
    for ticker in top100_tickers:
        print(f'{ticker}')
        try:
            # Try to get the trade signs and run lengths
            signs = trade_dict[ticker]
            L     = L_dict[ticker]['L']

            # Calculate autocorrelation and gamma
            taus, C = sign_autocorrelation(signs, max_lag = 10000)
            out     = estimate_gamma(taus, C, min_points = 1000, nbins = 10)

            # Estimate alpha
            alpha, xmin = estimate_alpha(L)

            # Append results
            results.append({'year' : year, 'ticker' : ticker, 'xmin' : xmin, 'alpha' : alpha, 'gamma_nlls' : out['gamma_nlls']})

        except KeyError as e:
            print(f'Warning: missing data for {ticker} in {year}, skipping... ({e})')
        except Exception as e:
            print(f'Warning: error processing {ticker} in {year}, skipping... ({e})')

alpha_gamma_df = pd.DataFrame(results)

alpha_gamma_df.to_csv(f'alpha_gamma_df_{identifier}.csv', index = False)
b2.put_file(f'alpha_gamma_df_{identifier}.csv', 'test_data')

Processing year: 2023
LAB
PPH
Calculating best minimal value for power law fit


Fitting xmin: 100%|██████████| 45/45 [00:00<00:00, 147.83it/s]


FSR
Calculating best minimal value for power law fit


Fitting xmin: 100%|██████████| 54/54 [00:01<00:00, 50.99it/s]


OMU
Calculating best minimal value for power law fit


...

BOX
Calculating best minimal value for power law fit


Fitting xmin: 100%|██████████| 54/54 [00:00<00:00, 192.05it/s]


ANH
Calculating best minimal value for power law fit


Fitting xmin: 100%|██████████| 47/47 [00:00<00:00, 122.39it/s]


CPU times: user 47min 53s, sys: 26 s, total: 48min 19s
Wall time: 48min 18s
